In [33]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import train_test_split

In [28]:
PATH_CSV_LABELS = '../tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../../Nuovi Dati/TemplateGeneratiClean'
PATH_MODELLO = '../../Modelli-v2/modello_pesi_inizializzati_polvere'
PATH_MODELLO_FINALE_RIADDESTRATO = '../../Modelli-v2/modello_finale_riaddestrato_polvere'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

HUBER_DELTA = 1.0

In [29]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,1_M_1,0.823433,64.257481
1,1_M_10,0.901885,73.084762
2,1_M_11,0.893868,72.182680
3,1_M_12,0.909014,73.886902
4,1_M_13,0.861603,68.552254


In [31]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(f'X_train: {len(X_train)}')
print(f'X_test: {len(X_test)}')

In [ ]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

In [1]:
# model.fit(x=X_train, y=y_train, batch_size=1, epochs=20, validation_data=(X_cv, y_cv))

In [9]:
#model.predict(X)

In [8]:
#y

In [2]:
# model.save(PATH_MODELLO_FINALE)

## Addestramento nuovo modello con Huber loss

In [49]:
conv1_weights = model.layers[0].weights
dense1_weights = model.layers[3].weights
dense2_weights = model.layers[4].weights

new_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS), weights=conv1_weights),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', weights=dense1_weights),
    layers.Dense(1, weights=dense2_weights)
])

In [50]:
new_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=HUBER_DELTA), metrics=['mae'])

In [51]:
new_model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/20


2023-07-13 11:06:13.941014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 1.4676 - mae: 1.9220

2023-07-13 11:06:23.738445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 11s 126ms/step - loss: 1.4676 - mae: 1.9220 - val_loss: 6.1297 - val_mae: 6.6131
Epoch 2/20
82/82 [==============================] - 10s 125ms/step - loss: 1.7435 - mae: 2.2057 - val_loss: 6.2099 - val_mae: 6.6919
Epoch 3/20
82/82 [==============================] - 10s 125ms/step - loss: 1.9298 - mae: 2.3911 - val_loss: 4.8327 - val_mae: 5.2992
Epoch 4/20
82/82 [==============================] - 10s 126ms/step - loss: 1.4335 - mae: 1.8897 - val_loss: 4.7747 - val_mae: 5.2512
Epoch 5/20
82/82 [==============================] - 10s 125ms/step - loss: 1.2686 - mae: 1.7053 - val_loss: 4.8329 - val_mae: 5.2861
Epoch 6/20
82/82 [==============================] - 10s 125ms/step - loss: 0.9311 - mae: 1.3724 - val_loss: 4.8566 - val_mae: 5.3362
Epoch 7/20
82/82 [==============================] - 10s 126ms/step - loss: 0.9863 - mae: 1.4045 - val_loss: 5.3334 - val_mae: 5.8272
Epoch 8/20
82/82 [==============================] - 10s 126ms/step - loss: 1.192

In [7]:
#new_model.predict(X)

In [6]:
#y

In [52]:
new_model.save(PATH_MODELLO_FINALE_RIADDESTRATO)

INFO:tensorflow:Assets written to: Modelli/modello_finale_riaddestrato_v3/assets


INFO:tensorflow:Assets written to: Modelli/modello_finale_riaddestrato_v3/assets


### Import nuovo modello

In [4]:
new_model = tf.keras.models.load_model(PATH_MODELLO_FINALE_RIADDESTRATO)

Metal device set to: Apple M1 Pro


2023-07-13 10:26:56.552384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-13 10:26:56.552752: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [60]:
def avg(lst):
    return sum(lst) / len(lst)

In [61]:
avg(abs(new_model.predict(X_test) - y_test))

4/4 [==============================] - 1s 355ms/step


array([6.51102037])